In [ ]:
import spikeinterface.full as si
import matplotlib.pyplot as plt
import numpy as np
import probeinterface as pi
from pathlib import Path
import os 
import pandas as pd 

global_job_kwargs = dict(n_jobs=4, chunk_duration="1s",progress_bar=True)
si.set_global_job_kwargs(**global_job_kwargs)


basefolder="F:/copydaya/M7_1_copy"

metapath = basefolder + str('/Meta')
if not os.path.isdir(metapath):
   os.makedirs(metapath)


recording =  si.read_spikeglx(basefolder, stream_id='imec1.ap', load_sync_channel=False)
lfp = si.read_spikeglx(basefolder, stream_id='imec1.lf', load_sync_channel=False)
event =  si.read_spikeglx(basefolder, stream_id='nidq', load_sync_channel=False)
print(recording)

#recording = si.ChannelSliceRecording(recording, channel_ids=recording.get_channel_ids()[180:330])


In [ ]:


bad_channel_ids, channel_labels = si.detect_bad_channels(lfp,method = 'coherence+psd',outside_channels_location = 'both')
names = lfp.channel_ids
depth = lfp.get_channel_locations()[:,1]


ar = pd.DataFrame({'name':names, 'depth':depth, 'labels':channel_labels})
ar.to_csv(metapath + str('/lfp_labels.csv'))
filtered_ar = ar[ar['labels'] == 'out']

print(filtered_ar)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
si.plot_probe_map(recording4, ax=ax, with_channel_ids=True)
ax.set_ylim(-200,3000)

In [ ]:
rec1 = si.highpass_filter(recording, freq_min=400.)
rec1 = si.phase_shift(rec1)
bad_channel_ids, channel_labels = si.detect_bad_channels(rec1,method = 'coherence+psd')
print(bad_channel_ids)
rec1 = si.interpolate_bad_channels(recording=rec1, bad_channel_ids=bad_channel_ids)

rec1 = si.common_reference(rec1, operator="median", reference="global")
print(rec1)


%matplotlib widget
si.plot_traces({'raw':recording,'filtered':rec1}, backend='ipywidgets')

In [ ]:


def extract_and_save_ttl_events(data, bits, save_path):
    digital_signals = data.get_traces()
    digital_word = digital_signals[:, 0]
    sampling_rate = data.get_sampling_frequency()
    for bit in bits:
        # Extract TTL pulses for the current bit
        ttl_timestamps = extract_ttl_from_bit(digital_word, bit, sampling_rate)
        
        ttl_df = pd.DataFrame(ttl_timestamps, columns=['timestamps'])
        
        filename = f'ttl_{bit}.csv'
        
        ttl_df.to_csv(f"{save_path}/{filename}", index=False)
        print(f"Extracted TTL event timestamps for bit {bit} saved to {filename}")


def extract_ttl_from_bit(digital_word, bit, sampling_rate):
    # Extract the specific bit from the word (bit-shifting and masking)
    ttl_signal = (digital_word >> bit) & 1  # Right shift and mask to isolate the specific bit
    
    # Detect rising edges (0 -> 1 transitions)
    ttl_rising_edges = np.where(np.diff(ttl_signal) > 0)[0]
    
    # Convert sample indices to timestamps (in seconds)
    ttl_timestamps = ttl_rising_edges / sampling_rate
    
    return ttl_timestamps


bits_to_extract = [0, 1, 2]  
extract_and_save_ttl_events(event , bits_to_extract, metapath)

In [ ]:
from spikeinterface.sorters import installed_sorters
installed_sorters()
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
torch.cuda.get_device_name(0)

Sorting_KS4 = si.run_sorter(sorter_name="kilosort4", recording=rec1, folder=basefolder + str('/sorted'),remove_existing_folder=True)

In [ ]:
#Sorting_KS4 = si.read_kilosort(folder_path=basefolder + str('/sorted/sorter_output'))
analyzer = si.create_sorting_analyzer(Sorting_KS4, rec1, sparse=True, format="memory")



In [ ]:
analyzer.compute(['random_spikes', 'waveforms', 'templates', 'noise_levels','unit_locations','correlograms'],**global_job_kwargs)
analyzer.compute('spike_amplitudes')
analyzer.compute('principal_components', n_components = 5, mode="by_channel_local",**global_job_kwargs)

In [ ]:
metric_names=['firing_rate', 'presence_ratio', 'snr','isi_violation', 'amplitude_cutoff']
metrics = si.compute_quality_metrics(analyzer, metric_names=metric_names)


amplitude_cutoff_thresh = 0.1
isi_violations_ratio_thresh = 0.5
presence_ratio_thresh = 0.9


our_query = f"(amplitude_cutoff < {amplitude_cutoff_thresh}) & (isi_violations_ratio < {isi_violations_ratio_thresh}) & (presence_ratio > {presence_ratio_thresh})"

keep_units = metrics.query(our_query)
keep_unit_ids = keep_units.index.values
analyzer_clean = analyzer.select_units(keep_unit_ids, folder=basefolder +str('/analyzer_clean'), format='binary_folder')
print(analyzer)
print(analyzer_clean)


In [ ]:
si.plot_sorting_summary(sorting_analyzer=sorting_analyzer, curation=True, backend='spikeinterface_gui')

In [1]:
si.export_to_phy(analyzer_clean, output_folder=basefolder + str('/sorted/phy'),**global_job_kwargs)

NameError: name 'si' is not defined

In [2]:
!phy template-gui E:/Florian/Data/Opto_2/M5686_3_g0/sorted/phy/params.py

16:21:18.498 [W] model:667            Skipping spike waveforms that do not exist, they will be extracted on the fly from the raw data as needed.
16:24:15.780 [I] supervisor:711       Change metadata_group for clusters 0 to good.
16:24:33.124 [I] supervisor:711       Change metadata_group for clusters 1 to good.
16:24:41.683 [I] supervisor:711       Change metadata_group for clusters 2 to good.
16:24:50.557 [I] supervisor:711       Change metadata_group for clusters 3 to mua.
16:24:56.611 [I] supervisor:711       Change metadata_group for clusters 4 to good.
16:25:03.452 [I] supervisor:711       Change metadata_group for clusters 5 to mua.
16:25:09.843 [I] supervisor:711       Change metadata_group for clusters 6 to good.
16:25:15.395 [I] supervisor:711       Change metadata_group for clusters 7 to good.
16:25:19.988 [I] supervisor:711       Change metadata_group for clusters 8 to good.
16:25:22.836 [I] supervisor:711       Change metadata_group for clusters 9 to good.
16:25:26.196 [I] 

In [2]:
import spikeinterface.full as si
import matplotlib.pyplot as plt
import numpy as np
import probeinterface as pi
from pathlib import Path
import os 
import pandas as pd 

global_job_kwargs = dict(n_jobs=4, chunk_duration="1s",progress_bar=True)
si.set_global_job_kwargs(**global_job_kwargs)
analyzer = si.load_sorting_analyzer("E:/Florian/Data/batch3/M7_1/analyzer_clean")
sorting(


C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator IncrementalPCA from version 1.5.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


AttributeError: module 'spikeinterface.full' has no attribute 'sorting_analyzer'

In [27]:
tm = analyzer.compute('template_metrics')
dat = tm.data

5       0.00055
6      0.000707
9      0.000573
10     0.000637
11     0.000583
         ...   
261    0.000553
262     0.00071
263    0.000533
264    0.000547
265    0.000643
Name: peak_to_valley, Length: 172, dtype: object


In [49]:
import os
import spikeinterface.full as si  # Assuming `si` is the module for loading the analyzer

import pandas as pd  # Ensure Pandas is imported
import matplotlib.pyplot as plt
from pathlib import Path  # For modern path handling

def find_key_in_dict(d, key_to_find):
    for key, value in d.items():
        if key == key_to_find:
            # If the value is a Pandas Series, convert it to a list
            if isinstance(value, pd.Series):
                return value.tolist()
            return value
        elif isinstance(value, dict):
            result = find_key_in_dict(value, key_to_find)
            if result:
                return result
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    result = find_key_in_dict(item, key_to_find)
                    if result:
                        return result
    return None

# Base directory where subfolders are located
base_dir = Path("E:/Florian/Data/batch3")


# Initialize a list to store all peak-to-valley values across subfolders
all_peak_to_valley_ms = []
original_indices = []  # To store the original indices for each value

# Iterate through all subfolders
for subfolder in base_dir.iterdir():  # Using pathlib to iterate
    if subfolder.is_dir():
        try:
            # Construct the full path and normalize it
            analyzer_path = subfolder / "analyzer"
            analyzer_path = os.path.normpath(analyzer_path)  # Normalize the path
            print(analyzer_path)
            # Load the analyzer for the current subfolder
            analyzer = si.load_sorting_analyzer(str(analyzer_path))  # Convert to string if needed
            tm = analyzer.compute('template_metrics')
            dat = tm.data

            # Extract peak-to-valley data
            peak_to_valley_data = find_key_in_dict(dat['metrics'], 'peak_to_valley')
            
            # Check if peak_to_valley_data is found
            if peak_to_valley_data:
                # If it's a Pandas Series, convert it to a list
                if isinstance(peak_to_valley_data, pd.Series):
                    peak_to_valley_data = peak_to_valley_data.tolist()
                
                # Convert to milliseconds and append to the cumulative list
                peak_to_valley_ms = [value * 1000 for value in peak_to_valley_data]
                
                # Append the values and their original indices
                all_peak_to_valley_ms.extend(peak_to_valley_ms)
                original_indices.extend([f"{subfolder.name}_row_{i}" for i in range(len(peak_to_valley_ms))])
        
        except Exception as e:
            print(f"Error processing {subfolder}: {e}")

# Filter values below 0.425ms (450ms) from the list
filtered_peak_to_valley_ms = [
    (index, value) for index, value in zip(original_indices, all_peak_to_valley_ms) if value < 0.425
]

# Create a DataFrame from the filtered data, including the original indices
df_filtered = pd.DataFrame(filtered_peak_to_valley_ms, columns=["Original_Index", "Peak_to_Valley (ms)"])

# Display the DataFrame
print(df_filtered)

df_filtered.to_csv("C:/Users/deepl/Desktop/Batch3.csv")


E:\Florian\Data\batch2\SNA-129853_2024-03-18_17-26-58\sorted\waveforms_clean2
Error processing E:\Florian\Data\batch2\SNA-129853_2024-03-18_17-26-58: [Errno 2] No such file or directory: 'E:\\Florian\\batch2\\SNA-129853_2024-03-18_17-26-58\\sorted\\sorter_output\\spike_times.npy'
E:\Florian\Data\batch2\SNA-129853_2024-03-19_13-36-52\sorted\waveforms_clean2
Error processing E:\Florian\Data\batch2\SNA-129853_2024-03-19_13-36-52: [Errno 2] No such file or directory: 'E:\\Florian\\batch2\\SNA-129853_2024-03-19_13-36-52\\sorted\\sorter_output\\spike_times.npy'
E:\Florian\Data\batch2\SNA-129853_2024-03-20_13-47-43\sorted\waveforms_clean2
Error processing E:\Florian\Data\batch2\SNA-129853_2024-03-20_13-47-43: [Errno 2] No such file or directory: 'E:\\Florian\\batch2\\SNA-129853_2024-03-20_13-47-43\\sorted\\sorter_output\\spike_times.npy'
E:\Florian\Data\batch2\SNA-129854_2024-03-12_17-24-42\sorted\waveforms_clean2
Error processing E:\Florian\Data\batch2\SNA-129854_2024-03-12_17-24-42: [Errno 

C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\spikeinterface\core\base.py:1117: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.2 is recommended
  warnings.warn(


In [63]:
Sorting_KS4 = si.read_kilosort("E:/Florian/Data/batch2/SNA-129853_2024-03-18_17-26-58/sorted/sorter_output")
rec1 = si.read_openephys("E:/Florian/Data/batch2/SNA-129853_2024-03-18_17-26-58/Record Node 178")
import probeinterface as pi

# download probe
probe = pi.get_probe(manufacturer='cambridgeneurotech', probe_name='ASSY-77-H3')
# add wiring
probe.wiring_to_device('ASSY-77>Adpt.A64-Om32_2x-sm-NN>RHD2164')


# set probe in place, ie, modify the current recording
rec1.set_probe(probe, group_mode="by_shank", in_place=True)
                   
analyzer = si.create_sorting_analyzer(Sorting_KS4, rec1, sparse=True, format="memory")


estimate_sparsity:   0%|          | 0/1873 [00:00<?, ?it/s]

In [64]:
analyzer.compute( [ 'random_spikes','waveforms','templates'],**global_job_kwargs)
tm = analyzer.compute('template_metrics')
dat = tm.data

compute_waveforms:   0%|          | 0/1873 [00:00<?, ?it/s]

C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\scipy\stats\_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\scipy\stats\_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\scipy\stats\_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


IndentationError: expected an indented block after 'if' statement on line 4 (466281426.py, line 5)

In [100]:
import os
import spikeinterface.full as si  # Assuming `si` is the module for loading the analyzer

import pandas as pd  # Ensure Pandas is imported
import matplotlib.pyplot as plt
from pathlib import Path  # For modern path handling

def find_key_in_dict(d, key_to_find):
    for key, value in d.items():
        if key == key_to_find:
            # If the value is a Pandas Series, convert it to a list
            if isinstance(value, pd.Series):
                return value.tolist()
            return value
        elif isinstance(value, dict):
            result = find_key_in_dict(value, key_to_find)
            if result:
                return result
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    result = find_key_in_dict(item, key_to_find)
                    if result:
                        return result
    return None
base_dir = Path("E:/Florian/Data/batch1")
all_peak_to_valley_ms = []
original_indices = []  

for subfolder in base_dir.iterdir():  # Using pathlib to iterate
    if subfolder.is_dir():
        try:
            # Construct the path for sorting output (fixed)
            sorting_output_path = subfolder / "sorted/sorter_output"
            sorting_output_path = os.path.normpath(sorting_output_path)  # Normalize the path

            # Search for a subfolder containing 'Record Node' and a number
            record_node_folders = [f for f in subfolder.iterdir() if "Record Node" in f.name]
            if not record_node_folders:
                print(f"No 'Record Node' found in {subfolder}. Skipping...")
                continue
            record_node_folder = record_node_folders[0]
            record_node_path = os.path.normpath(record_node_folder)  # Normalize the path
            
            # Print paths for debugging
            print(f"Sorting Output Path: {sorting_output_path}")
            print(f"Record Node Path: {record_node_path}")

            # Load Sorting (Kilosort) data
            Sorting_KS4 = si.read_kilosort(str(sorting_output_path))  # Ensure path is passed as string

            # Load OpenEphys data
            rec1 = si.read_openephys(str(record_node_path))

            # Download probe
            probe = pi.get_probe(manufacturer='cambridgeneurotech', probe_name='ASSY-77-H3')
            
            # Add wiring to device
            probe.wiring_to_device('ASSY-77>Adpt.A64-Om32_2x-sm-NN>RHD2164')

            # Set the probe for the current recording
            rec1.set_probe(probe, group_mode="by_shank", in_place=True)

            # Create the sorting analyzer with the loaded data
            analyzer = si.create_sorting_analyzer(Sorting_KS4, rec1, sparse=True, format="memory")

            # Compute the necessary metrics
            global_job_kwargs = {}  # You can define the global arguments here if needed
            analyzer.compute(['random_spikes', 'waveforms', 'templates'], **global_job_kwargs)

            # Compute 'template_metrics'
            tm = analyzer.compute('template_metrics')
            dat = tm.data

            peak_to_valley_data = find_key_in_dict(dat['metrics'], 'peak_to_valley')
            
            # Check if peak_to_valley_data is found
            if peak_to_valley_data:
                # If it's a Pandas Series, convert it to a list
                if isinstance(peak_to_valley_data, pd.Series):
                    peak_to_valley_data = peak_to_valley_data.tolist()
                
                # Convert to milliseconds and append to the cumulative list
                peak_to_valley_ms = [value * 1000 for value in peak_to_valley_data]
                
                # Append the values and their original indices
                all_peak_to_valley_ms.extend(peak_to_valley_ms)
                original_indices.extend([f"{subfolder.name}_row_{i}" for i in range(len(peak_to_valley_ms))])
        
        except Exception as e:
                print(f"Error processing {subfolder}: {e}")

# Filter values below 0.425ms (450ms) from the list
filtered_peak_to_valley_ms = [
    (index, value) for index, value in zip(original_indices, all_peak_to_valley_ms) if value < 0.425
]

# Create a DataFrame from the filtered data, including the original indices
df_filtered = pd.DataFrame(filtered_peak_to_valley_ms, columns=["Original_Index", "Peak_to_Valley (ms)"])

# Display the DataFrame
print(df_filtered)

df_filtered.to_csv("C:/Users/deepl/Desktop/Batch1.csv")


Sorting Output Path: E:\Florian\Data\batch1\SNA-127307_2023-12-06_15-56-09\sorted\sorter_output
Record Node Path: E:\Florian\Data\batch1\SNA-127307_2023-12-06_15-56-09\Record Node 134


estimate_sparsity:   0%|          | 0/1593 [00:00<?, ?it/s]

compute_waveforms:   0%|          | 0/1593 [00:00<?, ?it/s]

C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\scipy\stats\_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\scipy\stats\_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\scipy\stats\_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


Sorting Output Path: E:\Florian\Data\batch1\SNA-127307_2023-12-07_15-42-45\sorted\sorter_output
Record Node Path: E:\Florian\Data\batch1\SNA-127307_2023-12-07_15-42-45\Record Node 134


...

Sorting Output Path: E:\Florian\Data\batch1\SNA-127316_2023-12-15_15-59-02\sorted\sorter_output
Record Node Path: E:\Florian\Data\batch1\SNA-127316_2023-12-15_15-59-02\Record Node 171
Error processing E:\Florian\Data\batch1\SNA-127316_2023-12-15_15-59-02: This dataset is multi-block. Spikeinterface can load one block at a time. Use 'block_index' to select the block to be loaded.
Sorting Output Path: E:\Florian\Data\batch1\SNA-127316_2023-12-16_14-41-01\sorted\sorter_output
Record Node Path: E:\Florian\Data\batch1\SNA-127316_2023-12-16_14-41-01\Record Node 171


estimate_sparsity:   0%|          | 0/1896 [00:00<?, ?it/s]

compute_waveforms:   0%|          | 0/1896 [00:00<?, ?it/s]

                            Original_Index  Peak_to_Valley (ms)
0     SNA-127307_2023-12-06_15-56-09_row_0             0.000000
1     SNA-127307_2023-12-06_15-56-09_row_2             0.000000
2     SNA-127307_2023-12-06_15-56-09_row_3             0.000000
3     SNA-127307_2023-12-06_15-56-09_row_4             0.000000
4     SNA-127307_2023-12-06_15-56-09_row_5             0.000000
..                                     ...                  ...
398  SNA-127316_2023-12-16_14-41-01_row_20             0.353333
399  SNA-127316_2023-12-16_14-41-01_row_23             0.040000
400  SNA-127316_2023-12-16_14-41-01_row_30             0.043333
401  SNA-127316_2023-12-16_14-41-01_row_49             0.350000
402  SNA-127316_2023-12-16_14-41-01_row_50             0.000000

[403 rows x 2 columns]


C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\scipy\stats\_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\scipy\stats\_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\Users\deepl\anaconda3\envs\si_env\lib\site-packages\scipy\stats\_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
